# ステップ5 バックプロパゲーションの理論

#### 連鎖律(チェインルール)とは
#### 複数の関数が連結した関数の微分が、それを構成する各関数の微分の積へと分解できることを表す。

#### 例題 : yをxで微分せよ

## $$y = (x^2+x+100)^3$$

$$$$

$$$$

$$$$

$$$$

#### (解答)

## $$u = x^2+x+100$$
## $$y = u^3$$

#### ここで連鎖律を利用して

## $$\frac{∂y}{∂x} = \frac{∂y}{∂u}\frac{∂u}{∂x}$$

## $$\frac{∂y}{∂x} = 3(x^2+x+100)^2(2x+1)$$

$$$$

$$$$

#### 図5-5

## $$y = C(B(A(x)))$$
## $$\frac{∂y}{∂x} =  ?$$

## $$b = B(A(x))$$
#### とおくと
## $$y = C(b)$$
## $$\frac{∂y}{∂b} = \frac{∂y}{∂y}\frac{∂y}{∂b}$$
#### と表せる

$$$$

$$$$

## $$a = A(x)$$
#### とおくと
## $$y = C(B(a))$$
## $$\frac{∂y}{∂a} = \frac{∂y}{∂y}\frac{∂y}{∂b}\frac{∂b}{∂a}$$
#### と表せる

$$$$

$$$$

## $$\frac{∂y}{∂x} = \frac{∂y}{∂y}\frac{∂y}{∂b}\frac{∂b}{∂a}\frac{∂a}{∂x}$$

## ステップ6 手作業によるバックプロパゲーション

In [13]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None # 微分値を持つ想定の変数を用意

In [14]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input # 関数が入力された変数を覚える
        return output
    
    def forward(self, x):
        raise NotImplementedError
    
    def backward(self, gy):
        raise NotImplementedError

In [15]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [16]:
import numpy as np

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [17]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [18]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
x.grad

3.297442541400256

## $$y = (e^{x^2})^2$$
## $$y' = 4xe^{2x^2}$$

In [11]:
4 * 0.5 * np.exp(0.5)

3.2974425414002564

## 演習問題ステップ6-1
#### 次のyについてx=0.5における数値微分を求めましょう。
## $$y = e^{e^{e^x}}$$
## $$y' = e^{(e^{e^x}+e^{x}+x)}$$

In [22]:
# 解答→以下をチャットに貼り付けてください








# print(x.grad)

## ステップ7

In [30]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None # 変数が自身を作り出した関数を覚える
    
    def set_creator(self, func):
        self.creator = func

In [31]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y) # outputを格納する変数を用意(inputとは別のVariableがいるイメージ)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

In [32]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [33]:
import numpy as np

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

## 演習問題Step7-1
#### 次の関数において、yのみを使ってxの値を出力しましょう。

In [37]:
x = Variable(np.array([0.5]))
f = Square()
y = f(x)

# 解答→以下をチャットに貼り付けてください



In [45]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

$$$$

$$$$

$$$$

In [46]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        f = self.creator
        if f is not None: # 入力値に辿り着くまで
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

In [47]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

In [48]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [49]:
import numpy as np

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [118]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
